In [1]:
import argparse
import functools
from collections import defaultdict

import numpy as np
import pandas as pd
import xgboost as xgb

from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics import log_loss
from sklearn.cross_validation import train_test_split

from xgboost import XGBClassifier

E:\Anaconda2\envs\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [90]:
X_train=pd.read_csv('../train/train_all_20170527.csv')
new_magic_train=pd.read_csv('../train/new_magic_train.csv')

y_train = pd.read_csv('../data/train.csv')['is_duplicate'].values

In [91]:
print(X_train.shape)
X_train['new_magic_train']=new_magic_train.q1_q2_wm_ratio
print(X_train.shape)
X_train.head()

(404290, 68)
(404290, 69)


,word_match,tfidf_wm,tfidf_wm_stops,jaccard,wc_diff,wc_ratio,wc_diff_unique,wc_ratio_unique,wc_diff_unq_stop,wc_ratio_unique_stop,...,z_word_len2,z_match_ratio,z_word_match,z_tfidf_sum1,z_tfidf_sum2,z_tfidf_mean1,z_tfidf_mean2,z_tfidf_len1,z_tfidf_len2,new_magic_train
0,0.727273,0.796024,0.772164,0.769231,2,0.857143,1,0.916667,1,0.833333,...,12,0.926829,0.727273,2.238763,2.086851,0.373127,0.417370,6,5,0.0
1,0.307692,0.359200,0.361758,0.250000,5,1.625000,4,1.500000,5,2.250000,...,13,0.661871,0.307692,2.200222,2.721306,0.440044,0.340163,5,8,0.0
2,0.363636,0.302555,0.355191,0.200000,4,0.714286,4,0.714286,1,0.833333,...,10,0.439394,0.363636,2.420729,2.203294,0.403455,0.440659,6,5,0.0
3,0.000000,0.000000,0.000000,0.000000,2,0.818182,1,0.900000,1,1.250000,...,9,0.086957,0.000000,1.722880,2.166837,0.574293,0.433367,3,5,0.0
4,0.000000,0.034253,0.000000,0.111111,6,0.538462,6,0.538462,5,0.500000,...,7,0.365217,0.000000,2.933787,1.985756,0.325976,0.496439,9,4,0.0


In [92]:
X_test=pd.read_csv('../test/test_all_20170527.csv')
new_magic_test=pd.read_csv('../test/new_magic_test.csv')

In [93]:
print(X_test.shape)
X_test['new_magic_train']=new_magic_test.q1_q2_wm_ratio
print(X_test.shape)
X_test.head()

(2345796, 68)
(2345796, 69)


,word_match,tfidf_wm,tfidf_wm_stops,jaccard,wc_diff,wc_ratio,wc_diff_unique,wc_ratio_unique,wc_diff_unq_stop,wc_ratio_unique_stop,...,z_word_len2,z_match_ratio,z_word_match,z_tfidf_sum1,z_tfidf_sum2,z_tfidf_mean1,z_tfidf_mean2,z_tfidf_len1,z_tfidf_len2,new_magic_train
0,0.266667,0.234251,0.274019,0.090909,3,1.272727,2,1.181818,3,1.500000,...,14,0.384000,0.266667,2.048751,2.831757,0.409750,0.314640,5,9,0.0
1,0.500000,0.436043,0.480962,0.235294,7,0.500000,7,0.500000,2,0.714286,...,7,0.440367,0.500000,2.209699,2.151824,0.441940,0.430365,5,5,0.0
2,0.444444,0.418727,0.468893,0.285714,8,0.428571,6,0.500000,3,0.500000,...,6,0.584270,0.444444,2.174504,1.715722,0.434901,0.571907,5,3,0.0
3,0.000000,0.000000,0.000000,0.000000,1,0.750000,1,0.750000,0,1.000000,...,3,0.545455,0.000000,1.337203,1.399856,0.668602,0.699928,2,2,0.0
4,0.800000,0.841273,1.000000,0.428571,2,1.500000,2,1.500000,1,0.666667,...,6,0.677419,0.800000,1.620889,1.399301,0.540296,0.699651,3,2,0.0


# Doc2vec Features

In [94]:
#Import Initial Packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
import gensim
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
import re 
from collections import namedtuple
import multiprocessing
import datetime
import os

tokenizer = RegexpTokenizer(r'\w+')
stopwords = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [8]:
model = gensim.models.doc2vec.Doc2Vec.load("../kernels/20170531_A_doc_2_vec_model1")

In [6]:
doc_train1=np.load("../train/doc_train1.npy")
doc_train2=np.load("../train/doc_train2.npy")
print(doc_train1.shape)
print(doc_train2.shape)

doc_test1=np.load("../test/doc_test1.npy")
doc_test2=np.load("../test/doc_test2.npy")
print(doc_test1.shape)
print(doc_test2.shape)

(404290, 100)
(404290, 100)
(2345796, 100)
(2345796, 100)


In [95]:
doc_cosine=[]
doc_cityblock=[]
doc_jaccard=[]
doc_canberra=[]
doc_euclidean=[]
doc_minkowski=[]
doc_braycurtis=[]

for i in range(404290):
    doc_cosine.append(cosine(doc_train1[i],doc_train2[i]))
    doc_cityblock.append(cityblock(doc_train1[i],doc_train2[i]))
    doc_jaccard.append(jaccard(doc_train1[i],doc_train2[i]))
    doc_canberra.append(canberra(doc_train1[i],doc_train2[i]))
    doc_euclidean.append(euclidean(doc_train1[i],doc_train2[i]))
    doc_minkowski.append(minkowski(doc_train1[i],doc_train2[i],3))   
    doc_braycurtis.append(braycurtis(doc_train1[i],doc_train2[i]))  

X_train['doc_cosine']=doc_cosine
X_train['doc_cityblock']=doc_cityblock
#X_train['doc_jaccard']=doc_jaccard
X_train['doc_canberra']=doc_canberra
X_train['doc_euclidean']=doc_euclidean
X_train['doc_minkowski']=doc_minkowski
X_train['doc_braycurtis']=doc_braycurtis

In [ ]:
new_magic_train['doc_cosine']=doc_cosine
new_magic_train['doc_cityblock']=doc_cityblock
#new_magic_train['doc_jaccard']=doc_jaccard
new_magic_train['doc_canberra']=doc_canberra
new_magic_train['doc_euclidean']=doc_euclidean
new_magic_train['doc_minkowski']=doc_minkowski
new_magic_train['doc_braycurtis']=doc_braycurtis

In [100]:
new_magic_train=new_magic_train.iloc[:,1:]
new_magic_train.to_csv('../train/train7'  + '.csv',index=False)

In [107]:
new_magic_train

,doc_cosine,doc_cityblock,doc_canberra,doc_euclidean,doc_minkowski,doc_braycurtis
0,0.090386,9.721020,32.920453,1.236991,0.692383,0.210261
1,0.387510,22.590268,57.143769,2.803876,1.516228,0.506834
2,0.334093,18.849759,52.245815,2.339814,1.257393,0.445929
3,0.589200,22.609040,61.910452,2.724312,1.441254,0.664584
4,0.354065,19.498236,52.409366,2.607866,1.515275,0.458009
5,0.267092,18.179259,50.444062,2.278378,1.228575,0.412288
6,0.812317,27.029154,64.875680,3.417202,1.830614,0.807429
7,0.223270,10.107093,41.601501,1.289897,0.716363,0.335654
8,0.053630,5.544379,27.019509,0.711520,0.388698,0.161625
9,0.358826,19.322576,54.478332,2.452727,1.337128,0.470585


In [101]:
doc_cosine=[]
doc_cityblock=[]
doc_jaccard=[]
doc_canberra=[]
doc_euclidean=[]
doc_minkowski=[]
doc_braycurtis=[]

for i in range(2345796):
    doc_cosine.append(cosine(doc_test1[i],doc_test2[i]))
    doc_cityblock.append(cityblock(doc_test1[i],doc_test2[i]))
    doc_jaccard.append(jaccard(doc_test1[i],doc_test2[i]))
    doc_canberra.append(canberra(doc_test1[i],doc_test2[i]))
    doc_euclidean.append(euclidean(doc_test1[i],doc_test2[i]))
    doc_minkowski.append(minkowski(doc_test1[i],doc_test2[i],3))   
    doc_braycurtis.append(braycurtis(doc_test1[i],doc_test2[i]))  

X_test['doc_cosine']=doc_cosine
X_test['doc_cityblock']=doc_cityblock
#X_test['doc_jaccard']=doc_jaccard
X_test['doc_canberra']=doc_canberra
X_test['doc_euclidean']=doc_euclidean
X_test['doc_minkowski']=doc_minkowski
X_test['doc_braycurtis']=doc_braycurtis

In [105]:
new_magic_test=pd.read_csv('../test/new_magic_test.csv')
new_magic_test['doc_cosine']=doc_cosine
new_magic_test['doc_cityblock']=doc_cityblock
#new_magic_train['doc_jaccard']=doc_jaccard
new_magic_test['doc_canberra']=doc_canberra
new_magic_test['doc_euclidean']=doc_euclidean
new_magic_test['doc_minkowski']=doc_minkowski
new_magic_test['doc_braycurtis']=doc_braycurtis
new_magic_test

,Unnamed: 0,q1_q2_wm_ratio,doc_cosine,doc_cityblock,doc_canberra,doc_euclidean,doc_minkowski,doc_braycurtis
0,0,0.00000,0.420033,21.305270,55.592162,2.726754,1.505659,0.515624
1,1,0.00000,0.306199,18.944344,52.468401,2.271785,1.198566,0.451845
2,2,0.00000,0.125974,9.791595,37.566120,1.219872,0.653386,0.258769
3,3,0.00000,0.277650,11.932618,48.705276,1.501782,0.826624,0.396395
4,4,0.00000,0.171409,10.386296,42.535025,1.309380,0.716089,0.314176
5,5,0.00000,0.550819,21.947090,58.218293,2.874877,1.626887,0.578341
6,6,0.89635,0.241203,14.392842,47.337792,1.767213,0.940215,0.377467
7,7,0.00000,0.457686,25.080092,53.950608,3.205892,1.770243,0.537089
8,8,0.00000,0.299435,14.620780,49.617429,1.837315,0.997151,0.420512
9,9,0.00000,0.367184,22.508866,54.294508,2.776293,1.482992,0.497826


In [112]:
new_magic_test=new_magic_test.iloc[:, 1:]
new_magic_test.to_csv('../test/test7'  + '.csv',index=False)

In [79]:
print(X_train.shape)
print(X_test.shape)

(404290, 75)
(2345796, 75)


# NOT TO RUN

In [8]:
low=['jaccard_distance','euclidean_distance','z_tfidf_len1','z_tfidf_len2','common_trigrams_len','common_bigrams_len']
low

['jaccard_distance',
 'euclidean_distance',
 'z_tfidf_len1',
 'z_tfidf_len2',
 'common_trigrams_len',
 'common_bigrams_len']

In [9]:
X_train.drop(low, axis=1,inplace=True)

In [82]:
X_train.head()

,word_match,tfidf_wm,tfidf_wm_stops,jaccard,wc_diff,wc_ratio,wc_diff_unique,wc_ratio_unique,wc_diff_unq_stop,wc_ratio_unique_stop,...,z_tfidf_mean2,z_tfidf_len1,z_tfidf_len2,new_magic_train,doc_cosine,doc_cityblock,doc_canberra,doc_euclidean,doc_minkowski,doc_braycurtis
272940,0.545455,0.502982,0.573567,0.400000,5,1.625000,5,1.625000,3,1.750000,...,0.375579,4,7,0.0,0.219847,14.302514,47.233899,1.762669,0.951646,0.370452
226468,0.800000,0.987288,1.000000,0.750000,2,0.750000,2,0.750000,1,0.666667,...,1.000000,2,1,0.0,0.263513,11.827588,46.593334,1.543885,0.866799,0.407985
188901,0.250000,0.304206,0.251230,0.357143,9,0.437500,5,0.583333,4,0.333333,...,0.706736,6,2,0.0,0.204556,14.180631,46.614406,1.780844,0.961549,0.375232
291430,0.800000,0.719539,0.731018,0.583333,1,1.111111,1,1.111111,0,1.000000,...,0.575335,3,3,0.0,0.058992,7.063419,28.853724,0.885246,0.478893,0.170643
116662,0.666667,0.635395,0.628306,0.545455,3,1.428571,3,1.428571,1,1.250000,...,0.440348,4,5,0.0,0.125739,11.256236,41.375654,1.366083,0.725301,0.274551


# REBALANCE DATA

In [80]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=4242)

#UPDownSampling
pos_train = X_train[y_train == 1]
neg_train = X_train[y_train == 0]
X_train = pd.concat((neg_train, pos_train.iloc[:int(0.8*len(pos_train))], neg_train))
y_train = np.array([0] * neg_train.shape[0] + [1] * pos_train.iloc[:int(0.8*len(pos_train))].shape[0] + [0] * neg_train.shape[0])
print(np.mean(y_train))
del pos_train, neg_train

pos_valid = X_valid[y_valid == 1]
neg_valid = X_valid[y_valid == 0]
X_valid = pd.concat((neg_valid, pos_valid.iloc[:int(0.8 * len(pos_valid))], neg_valid))
y_valid = np.array([0] * neg_valid.shape[0] + [1] * pos_valid.iloc[:int(0.8 * len(pos_valid))].shape[0] + [0] * neg_valid.shape[0])
print(np.mean(y_valid))
del pos_valid, neg_valid

0.189752932122
0.189234677675


In [83]:
X_train.iloc[:,1:]#566484 rows × 61 columns

,tfidf_wm,tfidf_wm_stops,jaccard,wc_diff,wc_ratio,wc_diff_unique,wc_ratio_unique,wc_diff_unq_stop,wc_ratio_unique_stop,same_start,...,z_tfidf_mean2,z_tfidf_len1,z_tfidf_len2,new_magic_train,doc_cosine,doc_cityblock,doc_canberra,doc_euclidean,doc_minkowski,doc_braycurtis
272940,0.502982,0.573567,0.400000,5,1.625000,5,1.625000,3,1.750000,1.0,...,0.375579,4,7,0.000000,0.219847,14.302514,47.233899,1.762669,0.951646,0.370452
226468,0.987288,1.000000,0.750000,2,0.750000,2,0.750000,1,0.666667,1.0,...,1.000000,2,1,0.000000,0.263513,11.827588,46.593334,1.543885,0.866799,0.407985
188901,0.304206,0.251230,0.357143,9,0.437500,5,0.583333,4,0.333333,1.0,...,0.706736,6,2,0.000000,0.204556,14.180631,46.614406,1.780844,0.961549,0.375232
291430,0.719539,0.731018,0.583333,1,1.111111,1,1.111111,0,1.000000,1.0,...,0.575335,3,3,0.000000,0.058992,7.063419,28.853724,0.885246,0.478893,0.170643
116662,0.635395,0.628306,0.545455,3,1.428571,3,1.428571,1,1.250000,1.0,...,0.440348,4,5,0.000000,0.125739,11.256236,41.375654,1.366083,0.725301,0.274551
43890,0.321936,0.356239,0.157895,2,0.833333,2,0.833333,1,0.833333,1.0,...,0.436867,6,5,0.000000,0.294514,15.459510,48.595959,1.994424,1.096964,0.397089
118319,0.950794,0.941124,0.950000,1,0.950000,1,0.950000,1,0.916667,1.0,...,0.279285,11,11,0.000000,0.021415,6.222304,21.475869,0.752624,0.398904,0.107864
128289,0.276998,0.321807,0.166667,1,0.950000,1,0.944444,2,0.800000,1.0,...,0.341302,9,7,0.211195,0.199951,15.471524,40.923988,2.043776,1.174952,0.305208
163915,0.451907,0.391941,0.416667,1,0.888889,1,0.888889,1,1.250000,0.0,...,0.482679,4,4,0.000000,0.123368,9.686178,36.439622,1.212388,0.645697,0.253946
187000,0.847361,0.818454,0.785714,1,0.928571,1,0.923077,1,0.875000,1.0,...,0.340110,8,8,0.000000,0.031427,5.790262,21.562028,0.724335,0.395043,0.121003


In [84]:
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 7
params['subsample'] = 0.6
params['base_score'] = 0.2
# params['scale_pos_weight'] = 0.2

d_train = xgb.DMatrix(X_train.iloc[:, 1:], label=y_train)
d_valid = xgb.DMatrix(X_valid.iloc[:, 1:], label=y_valid)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 3000, watchlist, early_stopping_rounds=50, verbose_eval=50)
print(log_loss(y_valid, bst.predict(d_valid)))
bst.save_model('../model/xgb_201705601_A' +'.mdl')

[0]	train-logloss:0.475175	valid-logloss:0.47455
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[50]	train-logloss:0.268204	valid-logloss:0.270145
[100]	train-logloss:0.217422	valid-logloss:0.220782
[150]	train-logloss:0.199916	valid-logloss:0.204235
[200]	train-logloss:0.192102	valid-logloss:0.197213
[250]	train-logloss:0.187512	valid-logloss:0.193293
[300]	train-logloss:0.183772	valid-logloss:0.190483
[350]	train-logloss:0.180262	valid-logloss:0.188202
[400]	train-logloss:0.177234	valid-logloss:0.186502
[450]	train-logloss:0.174758	valid-logloss:0.185286
[500]	train-logloss:0.172759	valid-logloss:0.184462
[550]	train-logloss:0.170781	valid-logloss:0.183694
[600]	train-logloss:0.168952	valid-logloss:0.18304
[650]	train-logloss:0.1672	valid-logloss:0.182447
[700]	train-logloss:0.165558	valid-logloss:0.181964
[750]	train-logloss:0.164088	valid-logloss:0.181611
[800]	train-logloss:0.16

In [85]:
print(X_test.shape)
d_test = xgb.DMatrix(X_test.iloc[:, 1:])
p_test = bst.predict(d_test)

(2345796, 75)


In [87]:
sub = pd.DataFrame()
sub['test_id'] =pd.read_csv('../data/test.csv')['test_id']
sub['is_duplicate'] = p_test
sub.to_csv('../output/xgb_20170601_A'  + '.csv',index=False)
print(sub[(sub.is_duplicate>0.5)].shape)

(75318, 2)


In [89]:
feature_score = bst.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
feature_score

[('tfidf_wm', 7326),
 ('tfidf_wm_stops', 5847),
 ('kur_q1vec', 5795),
 ('skew_q1vec', 5771),
 ('skew_q2vec', 5726),
 ('kur_q2vec', 5717),
 ('norm_wmd', 5188),
 ('new_magic_train', 5007),
 ('wmd', 4630),
 ('z_match_ratio', 4571),
 ('jaccard', 4502),
 ('z_tfidf_mean2', 4310),
 ('z_tfidf_mean1', 4217),
 ('z_tfidf_sum2', 4193),
 ('doc_cosine', 4184),
 ('doc_canberra', 4169),
 ('z_tfidf_sum1', 4120),
 ('fuzz_partial_ratio', 3973),
 ('fuzz_token_sort_ratio', 3523),
 ('doc_cityblock', 3398),
 ('q2_freq', 3228),
 ('doc_braycurtis', 3207),
 ('char_ratio', 3201),
 ('len_q2', 3186),
 ('common_unigrams_ratio', 3102),
 ('fuzz_partial_token_sort_ratio', 3070),
 ('doc_minkowski', 3020),
 ('len_q1', 2972),
 ('fuzz_token_set_ratio', 2945),
 ('canberra_distance', 2884),
 ('word_match.1', 2699),
 ('cosine_distance', 2693),
 ('q1_freq', 2523),
 ('doc_euclidean', 2433),
 ('char_diff_unq_stop', 2345),
 ('common_bigrams_ratio', 2301),
 ('char_diff', 2159),
 ('fuzz_qratio', 2097),
 ('len_char_q1', 1989),
 ('l

In [19]:
sub = pd.DataFrame()
sub['test_id'] =pd.read_csv('../data/test.csv')['test_id']
sub['is_duplicate'] = p_test
sub.to_csv('../output/xgb_20170528_A'  + '.csv',index=False)
print(sub[(sub.is_duplicate>0.5)].shape)

(79717, 2)


In [20]:
feature_score = bst.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
feature_score

[('tfidf_wm', 10865),
 ('kur_q1vec', 10360),
 ('kur_q2vec', 9917),
 ('skew_q1vec', 9660),
 ('skew_q2vec', 9468),
 ('tfidf_wm_stops', 8573),
 ('norm_wmd', 7921),
 ('z_match_ratio', 7335),
 ('wmd', 7186),
 ('z_tfidf_mean2', 7164),
 ('z_tfidf_sum1', 6929),
 ('z_tfidf_mean1', 6907),
 ('z_tfidf_sum2', 6829),
 ('jaccard', 6581),
 ('fuzz_partial_ratio', 6197),
 ('char_ratio', 5487),
 ('fuzz_token_sort_ratio', 5459),
 ('q2_freq', 5275),
 ('fuzz_partial_token_sort_ratio', 5082),
 ('len_q2', 5022),
 ('canberra_distance', 4819),
 ('len_q1', 4715),
 ('cosine_distance', 4593),
 ('fuzz_token_set_ratio', 4572),
 ('q1_q2_intersect', 4529),
 ('q1_freq', 4458),
 ('common_unigrams_ratio', 4328),
 ('common_bigrams_ratio', 3800),
 ('char_diff_unq_stop', 3729),
 ('word_match.1', 3557),
 ('minkowski_distance', 3403),
 ('len_char_q1', 3334),
 ('len_char_q2', 3320),
 ('char_diff', 3285),
 ('fuzz_qratio', 3113),
 ('wc_ratio', 3047),
 ('wc_ratio_unique', 2864),
 ('total_unique_words', 2750),
 ('cityblock_distanc